In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Load dataset
data = pd.read_csv(r'C:\Users\samar kumar\Downloads\archive (2)\A_Z Handwritten Data.csv')

# Preprocess data
X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

# Normalize the pixel values
X = X / 255.0

# Reshape data to fit the model
X = X.reshape(-1, 28, 28, 1)

# Binarize labels
lb = LabelBinarizer()
y = lb.fit_transform(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
# Create the model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(26, activation='softmax')  # 26 classes for A-Z alphabets
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)


C:\Users\samar kumar\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 35s 15ms/step - accuracy: 0.8338 - loss: 0.5858 - val_accuracy: 0.9791 - val_loss: 0.0751
Epoch 2/10
 197/2328 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step - accuracy: 0.9603 - loss: 0.1430

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_list = []

for train_index, val_index in kf.split(X):
    X_train_kf, X_val_kf = X[train_index], X[val_index]
    y_train_kf, y_val_kf = y[train_index], y[val_index]
    
    model.fit(X_train_kf, y_train_kf, epochs=10, batch_size=128, verbose=0)
    y_pred = model.predict(X_val_kf)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_val_kf, axis=1)
    accuracy_list.append(accuracy_score(y_true, y_pred_classes))

print(f'Cross-Validation Accuracy: {np.mean(accuracy_list)}')


2328/2328 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step
Cross-Validation Accuracy: 0.9974815411464626


In [ ]:
import cv2

# Define function to preprocess image
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (28, 28))
    image = image / 255.0
    image = image.reshape(1, 28, 28, 1)
    return image

# Define function to get class label
def get_class_label(prediction):
    return chr(np.argmax(prediction) + 65)  # Convert to corresponding alphabet

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Define region of interest (ROI) for prediction
    roi = frame[100:400, 100:400]
    cv2.imshow('ROI', roi)
    
    # Preprocess the ROI and predict
    roi_preprocessed = preprocess_image(roi)
    prediction = model.predict(roi_preprocessed)
    class_label = get_class_label(prediction)
    
    # Display the prediction on the frame
    cv2.putText(frame, class_label, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 3)
    cv2.imshow('Handwritten Alphabet Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'model' is not defined